# DCPP Project : Indian Artists [Group 26]  Phase 2

In [445]:
import pandas as pd
 
# initialise data of lists.
team = {'Name':['Karan Saket', 'Pooja Ayyalasomayajula', 'Vikram Tantri', 'Shashank Jha'],
        'PGID':[12110063, 12110099, 12110114, 12110023]}
team_df = pd.DataFrame(team)
team_df

,Name,PGID
0,Karan Saket,12110063
1,Pooja Ayyalasomayajula,12110099
2,Vikram Tantri,12110114
3,Shashank Jha,12110023


### Step 1: Preparing the functions relevant for the data extraction/collection

###### As we have gathered the main artist data we look to obtain attributes from the artist such as the year they were born, age, year of death, notable contribution, category of art domain, if they have had any awards, nationality etc. The below notebook looks to scrape from internal wiki links on the artist biography or info box  

In [422]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

#function to get the text data from the site 
def getTextData1(url):
    response = requests.get(url)
    pageContent = response.text
    doc = BeautifulSoup(pageContent, 'html.parser')
#     return doc
    if doc.find('table', class_='infobox biography vcard') is not None: 
        categoryDoc = doc.find('table', class_='infobox biography vcard')
        return categoryDoc
    elif doc.find('table', class_='infobox vcard') is not None: 
        categoryDoc = doc.find('table', class_='infobox vcard')
        return categoryDoc
    else:         
        return ""

#function to get title, URL and description data from a given URL   
def getData1(a):
    columnName=[]
    columnVal =[]
    try:
        for tr in a.find_all('tr'):
            if tr.find('th'):
                for i in tr.find('th').get('class'): 
                    if i == 'infobox-label':
                        columnName.append(tr.find('th').text)
            if tr.find('td'):
                try:
                    for i in tr.find('td').get('class'): 
                        if i == 'infobox-data':
                            columnVal.append(tr.find('td').text) 
                except: 
                    pass
    except:
        pass
    #         if tr.find('td'):
    #             columnVal.append(tr.find('td').text)
    DataDict = {'BoxName':columnName,'BoxValue':columnVal}
    return DataDict
    
#function to make a dataframe
def makedf1(dict):
    df = pd.DataFrame(dict)
    return df    

#function to unexplode the matrix in column form to a row format
def makeRow1(df,url):
    df = df.set_index('BoxName').T
    df['Url']= url
    return df


### Step 2: Preparing the loop for attribute collection in subsites 

In [423]:
#Reading all the artists names and getting their attributes with the wiki URLs
df = pd.read_csv('final_list_of_artists.csv')
df = df.fillna('')
# df
test = df['ArtistWikiPage'][:]
test

0         https://en.wikipedia.org/wiki/Amrita_Sher-Gil
1              https://en.wikipedia.org/wiki/Jamini_Roy
2        https://en.wikipedia.org/wiki/Kalipada_Ghoshal
3                  https://en.wikipedia.org/wiki/Maniam
4             https://en.wikipedia.org/wiki/Manishi_Dey
                            ...                        
92    https://en.wikipedia.org/wiki/Vasudeo_S._Gaitonde
93         https://en.wikipedia.org/wiki/Vivan_Sundaram
94                  https://en.wikipedia.org/wiki/Yantr
95      https://en.wikipedia.org/wiki/Sudarsan_Pattnaik
96             https://en.wikipedia.org/wiki/Saba_Hasan
Name: ArtistWikiPage, Length: 97, dtype: object

In [424]:
# Getting unique columns from BoxName to ppopulate all attribute tables
title = []
# b = pd.DataFrame(a) 
# a1
start_time = time.time()

for i in test:
    a1 = makedf1(getData1(getTextData1(i)))
    a2 = a1['BoxName']
    for j in range(0,len(a2)):
#         print(j,a2[j])
        title.append(a2[j])
#     print(i,a2)

end_time = time.time()
total_time = round(end_time-start_time,2)

BoxTitle = pd.DataFrame(title)
BoxTitle = BoxTitle.drop_duplicates()
BoxTitle.columns =['Name']
BoxTitle
# print(str(total_time)+" seconds elapsed fetching this data")

,Name
0,Born
1,Died
2,Nationality
3,Education
4,Known for
5,Spouse(s)
10,Notable work
11,Awards
16,Movement
33,Native name


In [425]:
#Getting a table for all the unique attributes
BoxTitle_tbl = BoxTitle.set_index('Name').T
BoxTitle_tbl
# BoxTitle.to_csv('title.csv')

Name,Born,Died,Nationality,Education,Known for,Spouse(s),Notable work,Awards,Movement,Native name,...,Style,Honours,Partner(s),Years active,Constituency,Political party,Citizenship,Organization,Patron(s),Genre


In [428]:
#Given the template for collection data is obtained, running loop for all 96 sites to obtain row wise data for all attributes 
test2 = df['ArtistWikiPage'][:]
test2
BoxData = BoxTitle_tbl

start_time = time.time() #to capture time of execution
for i in test2:
    a1 = makedf1(getData1(getTextData1(i)))
    b1 = makeRow1(a1,i)
    BoxData = BoxData.append(b1,ignore_index = True)

end_time = time.time()
total_time = round(end_time-start_time,2)

BoxData = BoxData.fillna('')  #to remove all NaN values
# BoxData= BoxData.set_index("Url", inplace = True)
BoxData #This has the final list of all Box related data from wiki having all artist attributes

,Born,Died,Nationality,Education,Known for,Spouse(s),Notable work,Awards,Movement,Native name,...,Honours,Partner(s),Years active,Constituency,Political party,Citizenship,Organization,Patron(s),Genre,Url
0,"(1913-01-30)30 January 1913Budapest, Kingdom o...","5 December 1941(1941-12-05) (aged 28)Lahore, P...",Hungarian British Indian,Grande ChaumiereÉcole des Beaux-Arts (1930–34),Painting,Viktor Egan,,,,,...,,,,,,,,,,https://en.wikipedia.org/wiki/Amrita_Sher-Gil
1,"(1887-04-14)14 April 1887Beliatore, Bankura di...",24 April 1972(1972-04-24) (aged 85),Indian,,Painting,,painter,Padma Bhushan (1955),,,...,,,,,,,,,,https://en.wikipedia.org/wiki/Jamini_Roy
2,"(1906-09-00)September 1906Howrah, Bengal Presi...","29 April 1995(1995-04-29) (aged 88)Howrah, Wes...",Indian,,Painting,,,,Modern Indian Art,,...,,,,,,,,,,https://en.wikipedia.org/wiki/Kalipada_Ghoshal
3,,,,,,,,,,,...,,,,,,,,,,https://en.wikipedia.org/wiki/Maniam
4,Bijoy Chandra Dey(1909-09-22)22 September 1909...,"(1966-01-31)31 January 1966Kolkata, India",Indian,,"Painting, Drawing",,,,,,...,,,,,,,,,,https://en.wikipedia.org/wiki/Manishi_Dey
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,1924Maharashtra,10 August 2001 (aged 77),Indian,Sir J. J. School of Art,Abstract painting,,,Rockefeller Fellowship 1964 Padma Shri 1971,Abstract Expressionism,,...,,,,,,,,,,https://en.wikipedia.org/wiki/Vasudeo_S._Gaitonde
93,,,,,,,,,,,...,,,,,,,,,,https://en.wikipedia.org/wiki/Vivan_Sundaram
94,Real name: unknownDate of birth: unknown,,,,GraffitiStreet artSocial commentary,,,,,,...,,,,,,,,,,https://en.wikipedia.org/wiki/Yantr
95,"(1977-04-15) 15 April 1977 (age 44)Puri, Odis...",,Indian,,,,,Padma Shri (2014)[1],,,...,,,,,,,,,,https://en.wikipedia.org/wiki/Sudarsan_Pattnaik


In [432]:
#Modifying the boxdata to change its index for merging with the main artist data
BoxData1 = BoxData
# BoxData1
BoxData1 = BoxData1.set_index("Url", inplace = True)
BoxData1

,Born,Died,Nationality,Education,Known for,Spouse(s),Notable work,Awards,Movement,Native name,...,Style,Honours,Partner(s),Years active,Constituency,Political party,Citizenship,Organization,Patron(s),Genre
Url,,,,,,,,,,,,,,,,,,,,,
https://en.wikipedia.org/wiki/Amrita_Sher-Gil,"(1913-01-30)30 January 1913Budapest, Kingdom o...","5 December 1941(1941-12-05) (aged 28)Lahore, P...",Hungarian British Indian,Grande ChaumiereÉcole des Beaux-Arts (1930–34),Painting,Viktor Egan,,,,,...,,,,,,,,,,
https://en.wikipedia.org/wiki/Jamini_Roy,"(1887-04-14)14 April 1887Beliatore, Bankura di...",24 April 1972(1972-04-24) (aged 85),Indian,,Painting,,painter,Padma Bhushan (1955),,,...,,,,,,,,,,
https://en.wikipedia.org/wiki/Kalipada_Ghoshal,"(1906-09-00)September 1906Howrah, Bengal Presi...","29 April 1995(1995-04-29) (aged 88)Howrah, Wes...",Indian,,Painting,,,,Modern Indian Art,,...,,,,,,,,,,
https://en.wikipedia.org/wiki/Maniam,,,,,,,,,,,...,,,,,,,,,,
https://en.wikipedia.org/wiki/Manishi_Dey,Bijoy Chandra Dey(1909-09-22)22 September 1909...,"(1966-01-31)31 January 1966Kolkata, India",Indian,,"Painting, Drawing",,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://en.wikipedia.org/wiki/Vasudeo_S._Gaitonde,1924Maharashtra,10 August 2001 (aged 77),Indian,Sir J. J. School of Art,Abstract painting,,,Rockefeller Fellowship 1964 Padma Shri 1971,Abstract Expressionism,,...,,,,,,,,,,
https://en.wikipedia.org/wiki/Vivan_Sundaram,,,,,,,,,,,...,,,,,,,,,,
https://en.wikipedia.org/wiki/Yantr,Real name: unknownDate of birth: unknown,,,,GraffitiStreet artSocial commentary,,,,,,...,,,,,,,,,,


In [433]:
# Creating a dataframe for the original list and changing its index to match with the unique URL available in attributes data too
final_df = df
final_df = final_df.rename(columns={'ArtistWikiPage':'Url'})
final_df.set_index("Url", inplace = True)
final_df

,ArtistName,ArtistOccupation,YearBorn,YearDied
Url,,,,
https://en.wikipedia.org/wiki/Amrita_Sher-Gil,Amrita Sher-Gil,,1913,1941
https://en.wikipedia.org/wiki/Jamini_Roy,Jamini Roy,,1887,1972
https://en.wikipedia.org/wiki/Kalipada_Ghoshal,Kalipada Ghoshal,,1906,1995
https://en.wikipedia.org/wiki/Maniam,Maniam,,1924,1968
https://en.wikipedia.org/wiki/Manishi_Dey,Manishi Dey,,1909,1966
...,...,...,...,...
https://en.wikipedia.org/wiki/Vasudeo_S._Gaitonde,Vasudeo S. Gaitonde,painter,1924,2001
https://en.wikipedia.org/wiki/Vivan_Sundaram,Vivan Sundaram,installation artist,1943,
https://en.wikipedia.org/wiki/Yantr,Yantr,,,


### Step 3:  Merging attributes with the main Artists data

In [434]:
#Merging the main artist data with the artist attribute data
df_merge = final_df.join(BoxData1)
df_merge = df_merge.reset_index()
df_merge = df_merge.rename(columns={'Url':'ArtistWikiLink'})
df_merge['Age'] = pd.to_numeric(df_merge['YearDied'])-pd.to_numeric(df_merge['YearBorn'])
df_merge = df_merge.fillna(0)
df_merge
# final_df
# BoxData1

,ArtistWikiLink,ArtistName,ArtistOccupation,YearBorn,YearDied,Born,Died,Nationality,Education,Known for,...,Honours,Partner(s),Years active,Constituency,Political party,Citizenship,Organization,Patron(s),Genre,Age
0,https://en.wikipedia.org/wiki/Amrita_Sher-Gil,Amrita Sher-Gil,,1913,1941,"(1913-01-30)30 January 1913Budapest, Kingdom o...","5 December 1941(1941-12-05) (aged 28)Lahore, P...",Hungarian British Indian,Grande ChaumiereÉcole des Beaux-Arts (1930–34),Painting,...,,,,,,,,,,28.0
1,https://en.wikipedia.org/wiki/Jamini_Roy,Jamini Roy,,1887,1972,"(1887-04-14)14 April 1887Beliatore, Bankura di...",24 April 1972(1972-04-24) (aged 85),Indian,,Painting,...,,,,,,,,,,85.0
2,https://en.wikipedia.org/wiki/Kalipada_Ghoshal,Kalipada Ghoshal,,1906,1995,"(1906-09-00)September 1906Howrah, Bengal Presi...","29 April 1995(1995-04-29) (aged 88)Howrah, Wes...",Indian,,Painting,...,,,,,,,,,,89.0
3,https://en.wikipedia.org/wiki/Maniam,Maniam,,1924,1968,,,,,,...,,,,,,,,,,44.0
4,https://en.wikipedia.org/wiki/Manishi_Dey,Manishi Dey,,1909,1966,Bijoy Chandra Dey(1909-09-22)22 September 1909...,"(1966-01-31)31 January 1966Kolkata, India",Indian,,"Painting, Drawing",...,,,,,,,,,,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,https://en.wikipedia.org/wiki/Vasudeo_S._Gaitonde,Vasudeo S. Gaitonde,painter,1924,2001,1924Maharashtra,10 August 2001 (aged 77),Indian,Sir J. J. School of Art,Abstract painting,...,,,,,,,,,,77.0
93,https://en.wikipedia.org/wiki/Vivan_Sundaram,Vivan Sundaram,installation artist,1943,,,,,,,...,,,,,,,,,,0.0
94,https://en.wikipedia.org/wiki/Yantr,Yantr,,,,Real name: unknownDate of birth: unknown,,,,GraffitiStreet artSocial commentary,...,,,,,,,,,,0.0
95,https://en.wikipedia.org/wiki/Sudarsan_Pattnaik,Sudarsan Pattnaik,Sand Artist,1977,,"(1977-04-15) 15 April 1977 (age 44)Puri, Odis...",,Indian,,,...,,,,,,,,,,0.0


In [435]:
#Listing columns to rearrange
cols = list(df_merge.columns.values)
cols

['ArtistWikiLink',
 'ArtistName',
 'ArtistOccupation',
 'YearBorn',
 'YearDied',
 'Born',
 'Died',
 'Nationality',
 'Education',
 'Known\xa0for',
 'Spouse(s)',
 'Notable work',
 'Awards',
 'Movement',
 'Native name',
 'Resting place',
 'Pen name',
 'Occupation',
 'Language',
 'Period',
 'Literary movement',
 'Notable works',
 'Notable awards',
 'Spouse',
 'Children',
 'Relatives',
 'Signature',
 'Other\xa0names',
 'Alma\xa0mater',
 'Website',
 'Parent(s)',
 'Style',
 'Honours',
 'Partner(s)',
 'Years\xa0active',
 'Constituency',
 'Political party',
 'Citizenship',
 'Organization',
 'Patron(s)',
 'Genre',
 'Age']

In [415]:
#rearranging as per convenience
cols2 = ['ArtistName',
 'YearBorn',
 'YearDied',
 'Age',
 'Born',
 'Died',
 'ArtistOccupation',
 'ArtistWikiLink',
 'Nationality',
 'Education',
 'Known\xa0for',
 'Spouse(s)',
 'Notable work',
 'Awards',
 'Movement',
 'Native name',
 'Resting place',
 'Pen name',
 'Occupation',
 'Language',
 'Period',
 'Literary movement',
 'Notable works',
 'Notable awards',
 'Spouse',
 'Children',
 'Relatives',
 'Signature',
 'Other\xa0names',
 'Alma\xa0mater',
 'Website',
 'Parent(s)',
 'Style',
 'Honours',
 'Partner(s)',
 'Years\xa0active',
 'Constituency',
 'Political party',
 'Citizenship',
 'Organization',
 'Patron(s)',
 'Genre']
cols2

['ArtistName',
 'YearBorn',
 'YearDied',
 'Age',
 'Born',
 'Died',
 'ArtistOccupation',
 'ArtistWikiLink',
 'Nationality',
 'Education',
 'Known\xa0for',
 'Spouse(s)',
 'Notable work',
 'Awards',
 'Movement',
 'Native name',
 'Resting place',
 'Pen name',
 'Occupation',
 'Language',
 'Period',
 'Literary movement',
 'Notable works',
 'Notable awards',
 'Spouse',
 'Children',
 'Relatives',
 'Signature',
 'Other\xa0names',
 'Alma\xa0mater',
 'Website',
 'Parent(s)',
 'Style',
 'Honours',
 'Partner(s)',
 'Years\xa0active',
 'Constituency',
 'Political party',
 'Citizenship',
 'Organization',
 'Patron(s)',
 'Genre']

In [436]:
#Dataset with rearranged columns  
df_merge2 = df_merge[cols2]
df_merge2 = df_merge2.fillna('0')
# df_merge2['YearBorn'] =  df_merge2['YearBorn'].astype(float)
df_merge2

,ArtistName,YearBorn,YearDied,Age,Born,Died,ArtistOccupation,ArtistWikiLink,Nationality,Education,...,Style,Honours,Partner(s),Years active,Constituency,Political party,Citizenship,Organization,Patron(s),Genre
0,Amrita Sher-Gil,1913,1941,28.0,"(1913-01-30)30 January 1913Budapest, Kingdom o...","5 December 1941(1941-12-05) (aged 28)Lahore, P...",,https://en.wikipedia.org/wiki/Amrita_Sher-Gil,Hungarian British Indian,Grande ChaumiereÉcole des Beaux-Arts (1930–34),...,,,,,,,,,,
1,Jamini Roy,1887,1972,85.0,"(1887-04-14)14 April 1887Beliatore, Bankura di...",24 April 1972(1972-04-24) (aged 85),,https://en.wikipedia.org/wiki/Jamini_Roy,Indian,,...,,,,,,,,,,
2,Kalipada Ghoshal,1906,1995,89.0,"(1906-09-00)September 1906Howrah, Bengal Presi...","29 April 1995(1995-04-29) (aged 88)Howrah, Wes...",,https://en.wikipedia.org/wiki/Kalipada_Ghoshal,Indian,,...,,,,,,,,,,
3,Maniam,1924,1968,44.0,,,,https://en.wikipedia.org/wiki/Maniam,,,...,,,,,,,,,,
4,Manishi Dey,1909,1966,57.0,Bijoy Chandra Dey(1909-09-22)22 September 1909...,"(1966-01-31)31 January 1966Kolkata, India",,https://en.wikipedia.org/wiki/Manishi_Dey,Indian,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Vasudeo S. Gaitonde,1924,2001,77.0,1924Maharashtra,10 August 2001 (aged 77),painter,https://en.wikipedia.org/wiki/Vasudeo_S._Gaitonde,Indian,Sir J. J. School of Art,...,,,,,,,,,,
93,Vivan Sundaram,1943,,0.0,,,installation artist,https://en.wikipedia.org/wiki/Vivan_Sundaram,,,...,,,,,,,,,,
94,Yantr,,,0.0,Real name: unknownDate of birth: unknown,,,https://en.wikipedia.org/wiki/Yantr,,,...,,,,,,,,,,
95,Sudarsan Pattnaik,1977,,0.0,"(1977-04-15) 15 April 1977 (age 44)Puri, Odis...",,Sand Artist,https://en.wikipedia.org/wiki/Sudarsan_Pattnaik,Indian,,...,,,,,,,,,,


### Step 4: Data Cleaning of attributes

In [437]:
#Cleaning Age data for people who are not dead yet

df_merge2['Age'] = pd.to_numeric(df_merge2['Age'], errors='coerce')  #Converting Age to numeric
df_merge2['YearBorn'] = pd.to_numeric(df_merge2['YearBorn'], errors='coerce') #Converting year s to numeric
df_merge2 = df_merge2.assign(Age_Cleaned=np.where(df_merge2.Age==0, 2021-df_merge2.YearBorn, df_merge2.Age)) #Creating a clean age column
# df_merge2 = df_merge2.fillna('')

df_merge2

,ArtistName,YearBorn,YearDied,Age,Born,Died,ArtistOccupation,ArtistWikiLink,Nationality,Education,...,Honours,Partner(s),Years active,Constituency,Political party,Citizenship,Organization,Patron(s),Genre,Age_Cleaned
0,Amrita Sher-Gil,1913.0,1941,28.0,"(1913-01-30)30 January 1913Budapest, Kingdom o...","5 December 1941(1941-12-05) (aged 28)Lahore, P...",,https://en.wikipedia.org/wiki/Amrita_Sher-Gil,Hungarian British Indian,Grande ChaumiereÉcole des Beaux-Arts (1930–34),...,,,,,,,,,,28.0
1,Jamini Roy,1887.0,1972,85.0,"(1887-04-14)14 April 1887Beliatore, Bankura di...",24 April 1972(1972-04-24) (aged 85),,https://en.wikipedia.org/wiki/Jamini_Roy,Indian,,...,,,,,,,,,,85.0
2,Kalipada Ghoshal,1906.0,1995,89.0,"(1906-09-00)September 1906Howrah, Bengal Presi...","29 April 1995(1995-04-29) (aged 88)Howrah, Wes...",,https://en.wikipedia.org/wiki/Kalipada_Ghoshal,Indian,,...,,,,,,,,,,89.0
3,Maniam,1924.0,1968,44.0,,,,https://en.wikipedia.org/wiki/Maniam,,,...,,,,,,,,,,44.0
4,Manishi Dey,1909.0,1966,57.0,Bijoy Chandra Dey(1909-09-22)22 September 1909...,"(1966-01-31)31 January 1966Kolkata, India",,https://en.wikipedia.org/wiki/Manishi_Dey,Indian,,...,,,,,,,,,,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Vasudeo S. Gaitonde,1924.0,2001,77.0,1924Maharashtra,10 August 2001 (aged 77),painter,https://en.wikipedia.org/wiki/Vasudeo_S._Gaitonde,Indian,Sir J. J. School of Art,...,,,,,,,,,,77.0
93,Vivan Sundaram,1943.0,,0.0,,,installation artist,https://en.wikipedia.org/wiki/Vivan_Sundaram,,,...,,,,,,,,,,78.0
94,Yantr,NaN,,0.0,Real name: unknownDate of birth: unknown,,,https://en.wikipedia.org/wiki/Yantr,,,...,,,,,,,,,,NaN
95,Sudarsan Pattnaik,1977.0,,0.0,"(1977-04-15) 15 April 1977 (age 44)Puri, Odis...",,Sand Artist,https://en.wikipedia.org/wiki/Sudarsan_Pattnaik,Indian,,...,,,,,,,,,,44.0


In [439]:
#Categorizing people who are dead or alive

df_merge2 = df_merge2.assign(Is_Dead=np.where(df_merge2.YearDied=='', 'Alive', 'Dead'))
df_merge2

,ArtistName,YearBorn,YearDied,Age,Born,Died,ArtistOccupation,ArtistWikiLink,Nationality,Education,...,Years active,Constituency,Political party,Citizenship,Organization,Patron(s),Genre,Age_Cleaned,Is_Indian,Is_Dead
0,Amrita Sher-Gil,1913.0,1941,28.0,"(1913-01-30)30 January 1913Budapest, Kingdom o...","5 December 1941(1941-12-05) (aged 28)Lahore, P...",,https://en.wikipedia.org/wiki/Amrita_Sher-Gil,Hungarian British Indian,Grande ChaumiereÉcole des Beaux-Arts (1930–34),...,,,,,,,,28.0,Other/Mixed,Dead
1,Jamini Roy,1887.0,1972,85.0,"(1887-04-14)14 April 1887Beliatore, Bankura di...",24 April 1972(1972-04-24) (aged 85),,https://en.wikipedia.org/wiki/Jamini_Roy,Indian,,...,,,,,,,,85.0,India,Dead
2,Kalipada Ghoshal,1906.0,1995,89.0,"(1906-09-00)September 1906Howrah, Bengal Presi...","29 April 1995(1995-04-29) (aged 88)Howrah, Wes...",,https://en.wikipedia.org/wiki/Kalipada_Ghoshal,Indian,,...,,,,,,,,89.0,India,Dead
3,Maniam,1924.0,1968,44.0,,,,https://en.wikipedia.org/wiki/Maniam,,,...,,,,,,,,44.0,Other/Mixed,Dead
4,Manishi Dey,1909.0,1966,57.0,Bijoy Chandra Dey(1909-09-22)22 September 1909...,"(1966-01-31)31 January 1966Kolkata, India",,https://en.wikipedia.org/wiki/Manishi_Dey,Indian,,...,,,,,,,,57.0,India,Dead
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Vasudeo S. Gaitonde,1924.0,2001,77.0,1924Maharashtra,10 August 2001 (aged 77),painter,https://en.wikipedia.org/wiki/Vasudeo_S._Gaitonde,Indian,Sir J. J. School of Art,...,,,,,,,,77.0,India,Dead
93,Vivan Sundaram,1943.0,,0.0,,,installation artist,https://en.wikipedia.org/wiki/Vivan_Sundaram,,,...,,,,,,,,78.0,Other/Mixed,Alive
94,Yantr,NaN,,0.0,Real name: unknownDate of birth: unknown,,,https://en.wikipedia.org/wiki/Yantr,,,...,,,,,,,,NaN,Other/Mixed,Alive
95,Sudarsan Pattnaik,1977.0,,0.0,"(1977-04-15) 15 April 1977 (age 44)Puri, Odis...",,Sand Artist,https://en.wikipedia.org/wiki/Sudarsan_Pattnaik,Indian,,...,,,,,,,,44.0,India,Alive


In [441]:
# Cleaning nationality to broader groupsto check if they are indian or not
df_merge2['Nationality'] =df_merge2['Nationality'].str.replace(' Indian', 'India')
df_merge2['Nationality'] =df_merge2['Nationality'].str.replace('Indian[1]', 'India')
df_merge2 = df_merge2.assign(Is_Indian=np.where(df_merge2.Nationality=='Indian', 'India', 'Other/Mixed'))
df_merge2

,ArtistName,YearBorn,YearDied,Age,Born,Died,ArtistOccupation,ArtistWikiLink,Nationality,Education,...,Constituency,Political party,Citizenship,Organization,Patron(s),Genre,Age_Cleaned,Is_Indian,Is_Dead,Award_Group
0,Amrita Sher-Gil,1913.0,1941,28.0,"(1913-01-30)30 January 1913Budapest, Kingdom o...","5 December 1941(1941-12-05) (aged 28)Lahore, P...",,https://en.wikipedia.org/wiki/Amrita_Sher-Gil,Hungarian British Indian,Grande ChaumiereÉcole des Beaux-Arts (1930–34),...,,,,,,,28.0,Other/Mixed,Dead,Other
1,Jamini Roy,1887.0,1972,85.0,"(1887-04-14)14 April 1887Beliatore, Bankura di...",24 April 1972(1972-04-24) (aged 85),,https://en.wikipedia.org/wiki/Jamini_Roy,Indian,,...,,,,,,,85.0,India,Dead,Padma Recepient
2,Kalipada Ghoshal,1906.0,1995,89.0,"(1906-09-00)September 1906Howrah, Bengal Presi...","29 April 1995(1995-04-29) (aged 88)Howrah, Wes...",,https://en.wikipedia.org/wiki/Kalipada_Ghoshal,Indian,,...,,,,,,,89.0,India,Dead,Other
3,Maniam,1924.0,1968,44.0,,,,https://en.wikipedia.org/wiki/Maniam,,,...,,,,,,,44.0,Other/Mixed,Dead,Other
4,Manishi Dey,1909.0,1966,57.0,Bijoy Chandra Dey(1909-09-22)22 September 1909...,"(1966-01-31)31 January 1966Kolkata, India",,https://en.wikipedia.org/wiki/Manishi_Dey,Indian,,...,,,,,,,57.0,India,Dead,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Vasudeo S. Gaitonde,1924.0,2001,77.0,1924Maharashtra,10 August 2001 (aged 77),painter,https://en.wikipedia.org/wiki/Vasudeo_S._Gaitonde,Indian,Sir J. J. School of Art,...,,,,,,,77.0,India,Dead,Padma Recepient
93,Vivan Sundaram,1943.0,,0.0,,,installation artist,https://en.wikipedia.org/wiki/Vivan_Sundaram,,,...,,,,,,,78.0,Other/Mixed,Alive,Other
94,Yantr,NaN,,0.0,Real name: unknownDate of birth: unknown,,,https://en.wikipedia.org/wiki/Yantr,,,...,,,,,,,NaN,Other/Mixed,Alive,Other
95,Sudarsan Pattnaik,1977.0,,0.0,"(1977-04-15) 15 April 1977 (age 44)Puri, Odis...",,Sand Artist,https://en.wikipedia.org/wiki/Sudarsan_Pattnaik,Indian,,...,,,,,,,44.0,India,Alive,Padma Recepient


In [442]:
#Cleaning Awards into broader groups based on whether they were a Padma recipient 
df_merge2 = df_merge2.assign(Award_Group=np.where(df_merge2.Awards.str.contains('Padma'), 'Padma Recipient', 'Other'))
df_merge2

,ArtistName,YearBorn,YearDied,Age,Born,Died,ArtistOccupation,ArtistWikiLink,Nationality,Education,...,Constituency,Political party,Citizenship,Organization,Patron(s),Genre,Age_Cleaned,Is_Indian,Is_Dead,Award_Group
0,Amrita Sher-Gil,1913.0,1941,28.0,"(1913-01-30)30 January 1913Budapest, Kingdom o...","5 December 1941(1941-12-05) (aged 28)Lahore, P...",,https://en.wikipedia.org/wiki/Amrita_Sher-Gil,Hungarian British Indian,Grande ChaumiereÉcole des Beaux-Arts (1930–34),...,,,,,,,28.0,Other/Mixed,Dead,Other
1,Jamini Roy,1887.0,1972,85.0,"(1887-04-14)14 April 1887Beliatore, Bankura di...",24 April 1972(1972-04-24) (aged 85),,https://en.wikipedia.org/wiki/Jamini_Roy,Indian,,...,,,,,,,85.0,India,Dead,Padma Recipient
2,Kalipada Ghoshal,1906.0,1995,89.0,"(1906-09-00)September 1906Howrah, Bengal Presi...","29 April 1995(1995-04-29) (aged 88)Howrah, Wes...",,https://en.wikipedia.org/wiki/Kalipada_Ghoshal,Indian,,...,,,,,,,89.0,India,Dead,Other
3,Maniam,1924.0,1968,44.0,,,,https://en.wikipedia.org/wiki/Maniam,,,...,,,,,,,44.0,Other/Mixed,Dead,Other
4,Manishi Dey,1909.0,1966,57.0,Bijoy Chandra Dey(1909-09-22)22 September 1909...,"(1966-01-31)31 January 1966Kolkata, India",,https://en.wikipedia.org/wiki/Manishi_Dey,Indian,,...,,,,,,,57.0,India,Dead,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Vasudeo S. Gaitonde,1924.0,2001,77.0,1924Maharashtra,10 August 2001 (aged 77),painter,https://en.wikipedia.org/wiki/Vasudeo_S._Gaitonde,Indian,Sir J. J. School of Art,...,,,,,,,77.0,India,Dead,Padma Recipient
93,Vivan Sundaram,1943.0,,0.0,,,installation artist,https://en.wikipedia.org/wiki/Vivan_Sundaram,,,...,,,,,,,78.0,Other/Mixed,Alive,Other
94,Yantr,NaN,,0.0,Real name: unknownDate of birth: unknown,,,https://en.wikipedia.org/wiki/Yantr,,,...,,,,,,,NaN,Other/Mixed,Alive,Other
95,Sudarsan Pattnaik,1977.0,,0.0,"(1977-04-15) 15 April 1977 (age 44)Puri, Odis...",,Sand Artist,https://en.wikipedia.org/wiki/Sudarsan_Pattnaik,Indian,,...,,,,,,,44.0,India,Alive,Padma Recipient


### Step 5: Exporting final data

In [421]:
df_merge2.to_csv('Final_IndianArtist_Data_wAttributes.csv',index=True)